Question 1 (10 points)
Perform a k-NN classification with all predictors except ID and ZIP code. Compute and report the accuracies (also called correct rates) in the validation set for odd k’s up to 19 (i.e., k = 1, 3, …, 19). What is the best choice of k?

Question 2 (7 points)
Using the best k, make predictions in the validation set. Based on the numbers in the classification matrix (also called a confusion matrix), explain how are the sensitivity and specificity calculated.

Question 3 (6 points)
Classify a new customer with the following profile: Age = 40, Experience = 10, Income = 84, Family = 2, CCAvg = 2, Education = 2, Mortgage = 0, Securities.Account = 0, CD.Account = 0, Online = 1, CreditCard = 1.

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import statsmodels.formula.api as sm
from pathlib import Path
from utilities import regressionSummary, printConfusionMatrix
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import roc_curve, auc
%matplotlib inline

#import data and organize table for easier usage
#DATA = Path('.').resolve().parent/'data'
bank_df = pd.read_csv("../'data'/'UniversalBank.csv'")
bank_df.drop(columns = ['ID', 'ZIP Code'], inplace = True)
columns = list(bank_df.columns)
columns.remove('Personal Loan')
columns.append('Personal Loan')
bank_df = bank_df[columns]
bank_df.columns = [s.strip().replace(' ', '_') for s in bank_df.columns]
bank_df.columns

In [14]:
trainData = bank_df.sample(frac=0.6, random_state=12345)
validData = bank_df.drop(trainData.index)
accuracy_df = pd.DataFrame({'k':range(1,20, 2), 'accuracy': 0})
scaler = preprocessing.StandardScaler()
scaler.fit(trainData.iloc[:, :-1]) #preprocess data except response column
bankNorm = pd.concat([pd.DataFrame(scaler.transform(bank_df.iloc[:,:-1]), 
                                   columns=bank_df.columns[:-1]), bank_df.loc[:,'Personal_Loan']], axis = 1)
trainNorm = bankNorm.iloc[trainData.index]
validNorm = bankNorm.iloc[validData.index]
for idx, row in accuracy_df.iterrows():
    knn = KNeighborsClassifier(n_neighbors=row.k).fit(trainNorm.iloc[:,:-1],
                                                     trainNorm.loc[:,'Personal_Loan'])
    knnPred = knn.predict(validNorm.iloc[:,:-1])
    accuracy_df.loc[idx, 'accuracy'] = accuracy_score(validNorm.loc[:,'Personal_Loan'], knnPred)
accuracy_df

,k,accuracy
0,1,0.9605
1,3,0.9660
2,5,0.9605
3,7,0.9595
4,9,0.9545
5,11,0.9550
6,13,0.9500
7,15,0.9465
8,17,0.9455
9,19,0.9435


Question 1 : the best choice of k = 3

In [13]:
#fit using k=3 and predict, create confusion matrix
knn = KNeighborsClassifier(n_neighbors=3).fit(trainNorm.iloc[:,:-1],
                                                     trainNorm.loc[:,'Personal_Loan'])
knnPred = knn.predict(validNorm.iloc[:,:-1])
printConfusionMatrix(knnPred, validNorm.iloc[:,-1:], 
                     class_names=['Personal_Loan(0)', 'Personal_Loan(1)'])


Confusion Matrix (Accuracy 0.9660)

                Reference
      Prediction Personal_Loan(0) Personal_Loan(1)
Personal_Loan(0)             1792               60
Personal_Loan(1)                8              140


Question 2 :
TP = 124, FP = 9, TN = 1792, FN = 75
Sensitivity = TP / (TP + FN) = 124/(124+75) = 62.3%
Specificity = TN / (FP + TN) = 1792/(9+1792 = 99.5%

In [10]:
newCustomer = pd.DataFrame([{'Age' : 40, 'Experience' : 10, 'Income' : 84, 'Family' : 2, 
                              'CCAvg' : 2, 'Education' : 2, 'Mortgage' : 0, 'Securities_Account' : 0, 
                              'CD_Account' : 0, 'Online' : 1, 'CreditCard' : 1}])
newCustomerNorm = pd.DataFrame(scaler.transform(newCustomer), columns=bank_df.columns[:-1])
knn_new = NearestNeighbors(n_neighbors = 3)
knn_new.fit(trainNorm.iloc[:,:-1])
distances, indices = knn_new.kneighbors(newCustomerNorm)
trainNorm.iloc[indices[0], :]

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities_Account,CD_Account,Online,CreditCard,Personal_Loan
3830,-1.015670,-1.082925,-0.865352,-0.365793,0.039522,1.353577,-0.558498,2.92996,-0.257831,0.821609,-0.635482,0
1978,-0.754386,-0.821412,-0.909458,-0.365793,-0.305947,1.353577,-0.558498,2.92996,-0.257831,0.821609,-0.635482,0
3569,-0.406007,-0.472728,-1.085882,-0.365793,-0.651417,1.353577,-0.558498,2.92996,-0.257831,0.821609,-0.635482,0


Question 3 : Classify the new customer as Personal Loan = 0

In [12]:
knn.predict_proba(newCustomerNorm)

array([[1., 0.]])